In [2]:
import json
import pandas as pd
import numpy as np
import csv
import os
import ast

import nltk
import gensim

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from wordcloud import WordCloud

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

C:\Users\sebre\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sebre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sebre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sebre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# test classification dataset
from collections import Counter
from sklearn.datasets import make_classification

# evaluate multinomial logistic regression model
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

In [4]:
firehouse = pd.read_csv('firehouse_with_reviews.csv', sep = ',')
subway = pd.read_csv('subway_with_reviews.csv', sep = ',')
jmike = pd.read_csv('jersey_mike_with_reviews.csv', sep = ',')
jj = pd.read_csv('jimmy_johns_with_reviews.csv', sep = ',')
potbelly = pd.read_csv('potbelly_with_reviews.csv', sep = ',')
quiznos = pd.read_csv('quiznos_with_reviews.csv', sep = ',')

In [5]:
firehouse.head()
subway.head()

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,...,categories,hours,review_id,user_id,stars_y,useful,funny,cool,text,date
0,0,VZzivJEbmHItOxNXJB3SpA,Subway,"330 Franklin Rd, Ste 900-d",Brentwood,TN,37027,36.026363,-86.789473,2.5,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ...",WBq2S1dhTme48WSYtk-Mkg,fVup-p6mgIMuWzNIj-CS1Q,1,0,0,0,"Do not eat here. First of all, it took them ab...",12/4/2016 22:28
1,1,VZzivJEbmHItOxNXJB3SpA,Subway,"330 Franklin Rd, Ste 900-d",Brentwood,TN,37027,36.026363,-86.789473,2.5,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ...",f-0P4HgQvtFR4OQgT-kFqQ,Lw6rx2yyzUbnFMUiD-Mk3Q,5,1,0,0,"Great food, clean establishment, and quick ser...",4/22/2017 6:24
2,2,VZzivJEbmHItOxNXJB3SpA,Subway,"330 Franklin Rd, Ste 900-d",Brentwood,TN,37027,36.026363,-86.789473,2.5,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ...",8OCbRtYMBu3sm8jZXMQxiA,Fh4gvyRoRjtYBhdTL6dDBg,2,1,0,0,Been standing here 15 min now. Three people ah...,3/23/2014 22:19
3,3,VZzivJEbmHItOxNXJB3SpA,Subway,"330 Franklin Rd, Ste 900-d",Brentwood,TN,37027,36.026363,-86.789473,2.5,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ...",89u5jcXM5M9zKLxzhgILkQ,KXDiP5LvmOIAH2M7rQyquw,2,0,0,0,Went over to Subway last evening their were 2 ...,1/11/2016 8:33
4,4,VZzivJEbmHItOxNXJB3SpA,Subway,"330 Franklin Rd, Ste 900-d",Brentwood,TN,37027,36.026363,-86.789473,2.5,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ...",Co8Kuvrsb_tiw4COwt3Izw,fVup-p6mgIMuWzNIj-CS1Q,1,0,0,0,Tried it again. Big mistake. I thought 2017 mi...,3/6/2017 22:30


In [6]:
frames = [firehouse, subway, jmike, jj, potbelly, quiznos]
df = pd.concat(frames)

In [7]:
df.tail()

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,...,categories,hours,review_id,user_id,stars_y,useful,funny,cool,text,date
319,319,xx8KU3Dp0heIDOcIzXOewQ,Quiznos,725 Campus Ct,Hazelwood,MO,63042,38.775479,-90.38834,4.5,...,"Restaurants, Sandwiches, Delis, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-19:0', ...",pfeVcCj_HOe9bK3nN5K82g,Icoqtxs4WZGdWYxhkSjovQ,5,0,0,0,From what I know this is family owned and oper...,2018-11-03 18:16:24
320,320,xx8KU3Dp0heIDOcIzXOewQ,Quiznos,725 Campus Ct,Hazelwood,MO,63042,38.775479,-90.38834,4.5,...,"Restaurants, Sandwiches, Delis, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-19:0', ...",CRwkpCKg8q3yN8B3sSdFSA,x8_WrbvIgqZ8-5BtJofc2Q,2,0,2,0,I use to go to the Quiznos in st. Charles on a...,2017-09-02 01:13:52
321,321,xx8KU3Dp0heIDOcIzXOewQ,Quiznos,725 Campus Ct,Hazelwood,MO,63042,38.775479,-90.38834,4.5,...,"Restaurants, Sandwiches, Delis, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-19:0', ...",OkvyYXHhjqUWX1pAmobSxw,_V4EtVZ5KlCFIZ4U6nN7XQ,5,0,0,0,If you want the best sandwich experience you'v...,2020-02-06 21:45:19
322,322,xx8KU3Dp0heIDOcIzXOewQ,Quiznos,725 Campus Ct,Hazelwood,MO,63042,38.775479,-90.38834,4.5,...,"Restaurants, Sandwiches, Delis, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-19:0', ...",5A-7dzFGxCUHeH5st8XEYg,TESUeCPNTpoovu7sJypdzw,5,1,0,0,"Oh Quizno's, how I missed you.\nThis restauran...",2013-05-16 00:31:16
323,323,xx8KU3Dp0heIDOcIzXOewQ,Quiznos,725 Campus Ct,Hazelwood,MO,63042,38.775479,-90.38834,4.5,...,"Restaurants, Sandwiches, Delis, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-19:0', ...",jPE_eLIoe3huZuQIGrYyaA,CQDg6a4wDgkqRUJZSzXc-w,5,1,0,0,I absolutely LOVE Quiznos. I will drive 30 min...,2019-07-02 06:24:22


In [8]:
# merge reviews texts by business_id in df_merge
df_merge = df.groupby(['business_id'], as_index=False).first()

In [9]:
df_merge

,business_id,Unnamed: 0,name,address,city,state,postal_code,latitude,longitude,stars_x,...,categories,hours,review_id,user_id,stars_y,useful,funny,cool,text,date
0,#NAME?,336,Subway,1 Brookings Dr,Saint Louis,MO,63105,38.647482,-90.309581,2.0,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ...",MYctGuflZVL9ctEF1--9qQ,0MHq2xX2O2WUbbW2BMwyhw,1,0,0,0,I repeated my order three times loudly and you...,11/21/2017 1:15
1,-2CPhK6ik9ZBgFX_F-dkxQ,3255,Subway,"14961 N Florida Ave, Ste 14961",Tampa,FL,33613,28.087029,-82.458310,2.0,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '8:0-21:30', ...",I6svHqxWhJnkQDkTKUPvbA,3DbINcPH5PI0DGEQp5mwbA,4,0,0,0,Just your typical subway. This one is particul...,10/24/2013 17:38
2,-7UDKbg_8TL4LVuYR6Ooyw,2953,Subway,"2451 Mcmullen Booth Rd, Ste 1",Clearwater,FL,33759,28.010475,-82.709550,3.0,...,"Sandwiches, Fast Food, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '9:0-22:0', '...",vI40HCh41z4-m5xI5Eog_w,V7-EP1CbhjQeEq7f6ab3DQ,4,1,0,1,Went here tonight and had a very nice experien...,2/28/2015 2:48
3,-bcnY0PELKRomT2z7eGQ3w,2048,Jersey Mike's Subs,575 Horsham Rd,Horsham,PA,19044,40.187752,-75.147294,3.5,...,"Delis, Sandwiches, Restaurants, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",PuDhaY19Jm4nIjcUh1GzBw,RlX07vMSZNsIhx2WOcO6wQ,5,0,1,0,One bite and I'm hooked! Friendly energetic st...,2016-10-01 17:27:49
4,-dEuJqTHuE6rv8MrndqoXA,1784,Jimmy John's,659 W Brandon Blvd,Brandon,FL,33511,27.937170,-82.293415,3.0,...,"Sandwiches, Food, Restaurants, Fast Food, Food...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",W6z_xkFWpBs3cIR7x_qdgQ,CsknEPODx4mjx1GQBUkMsw,4,0,0,0,"Pretty good, their bread is kinda hit or miss,...",2016-10-26 17:17:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,zQB2BH0NRx8r8kvDQXZ8mQ,2718,Jimmy John's,40715 US Hwy 19 N,Tarpon Springs,FL,34689,28.142290,-82.739463,4.0,...,"Fast Food, Food Delivery Services, Food, Sandw...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",pl5KBp9RCRPm45RYMkQCIw,Um5bfs5DH6eizgjH3xZsvg,3,0,0,0,"My first time into a Jimmy Johns, and I'm glad...",2014-08-10 20:38:49
830,zRjpagNQNZoovYfnmKxYmw,1085,Subway,"1084 E Lancaster Ave, Ste 1",Bryn Mawr,PA,19010,40.026373,-75.328650,3.0,...,"Restaurants, Fast Food, Sandwiches","{'Monday': '7:0-22:0', 'Tuesday': '7:0-22:0', ...",XKDQ-2ex4mBNGgxf7xAhfg,eFUcc2QbaW0InWgsBsMiwA,3,1,0,0,I've gotten a few sandwiches from here. It's ...,8/4/2016 14:02
831,zciO1XFzcKlX98Bi3XOGww,625,Subway,"3901 N Belt W, Ste B",Bellville,IL,62226,38.541845,-90.013896,1.5,...,"Restaurants, Sandwiches, Fast Food","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...",XgFyQatlwTUj2y4nDTDk2g,#NAME?,1,0,0,0,HORRIBLE!! GO TO ANOTHER SUBWAY NOT THIS ONE!...,2/25/2019 23:26
832,zqsmGvH1aPMDFJB-jFNPwQ,3689,Jimmy John's,15069 Manchester Rd,Ballwin,MO,63011,38.593303,-90.549531,1.5,...,"Delis, Fast Food, Restaurants, Food, Sandwiche...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",aXTdF9clU0cTqgAEKTt8KA,hGJ7kvBRdiiUscHsirlxbQ,1,0,0,0,After 30 minutes I had no food. Called up ther...,2017-11-18 21:51:42


In [10]:
df_review = df_merge[['business_id', 'name', 'address', 'stars_x', 'text']]
df_review

,business_id,name,address,stars_x,text
0,#NAME?,Subway,1 Brookings Dr,2.0,I repeated my order three times loudly and you...
1,-2CPhK6ik9ZBgFX_F-dkxQ,Subway,"14961 N Florida Ave, Ste 14961",2.0,Just your typical subway. This one is particul...
2,-7UDKbg_8TL4LVuYR6Ooyw,Subway,"2451 Mcmullen Booth Rd, Ste 1",3.0,Went here tonight and had a very nice experien...
3,-bcnY0PELKRomT2z7eGQ3w,Jersey Mike's Subs,575 Horsham Rd,3.5,One bite and I'm hooked! Friendly energetic st...
4,-dEuJqTHuE6rv8MrndqoXA,Jimmy John's,659 W Brandon Blvd,3.0,"Pretty good, their bread is kinda hit or miss,..."
...,...,...,...,...,...
829,zQB2BH0NRx8r8kvDQXZ8mQ,Jimmy John's,40715 US Hwy 19 N,4.0,"My first time into a Jimmy Johns, and I'm glad..."
830,zRjpagNQNZoovYfnmKxYmw,Subway,"1084 E Lancaster Ave, Ste 1",3.0,I've gotten a few sandwiches from here. It's ...
831,zciO1XFzcKlX98Bi3XOGww,Subway,"3901 N Belt W, Ste B",1.5,HORRIBLE!! GO TO ANOTHER SUBWAY NOT THIS ONE!...
832,zqsmGvH1aPMDFJB-jFNPwQ,Jimmy John's,15069 Manchester Rd,1.5,After 30 minutes I had no food. Called up ther...


## NLP Text Clean

In [44]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english') #stopwords.append("n't")

In [134]:
len(stopwords)

179

In [135]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [16]:
# Load the regular expression library
import re

# Remove punctuation
df_review['text_processed'] = \
df_review['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df_review['text_processed'] = \
df_review['text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
df_review['text_processed'].tail()

C:\Users\sebre\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\sebre\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


829    my first time into a jimmy johns and i'm glad ...
830    i've gotten a few sandwiches from here  it's p...
831    horrible  go to another subway not this one  i...
832    after 30 minutes i had no food called up there...
833    this place rocks love them subs get over there...
Name: text_processed, dtype: object

In [17]:
df_review

,business_id,name,address,stars_x,text,text_processed
0,#NAME?,Subway,1 Brookings Dr,2.0,I repeated my order three times loudly and you...,i repeated my order three times loudly and you...
1,-2CPhK6ik9ZBgFX_F-dkxQ,Subway,"14961 N Florida Ave, Ste 14961",2.0,Just your typical subway. This one is particul...,just your typical subway this one is particula...
2,-7UDKbg_8TL4LVuYR6Ooyw,Subway,"2451 Mcmullen Booth Rd, Ste 1",3.0,Went here tonight and had a very nice experien...,went here tonight and had a very nice experien...
3,-bcnY0PELKRomT2z7eGQ3w,Jersey Mike's Subs,575 Horsham Rd,3.5,One bite and I'm hooked! Friendly energetic st...,one bite and i'm hooked friendly energetic sta...
4,-dEuJqTHuE6rv8MrndqoXA,Jimmy John's,659 W Brandon Blvd,3.0,"Pretty good, their bread is kinda hit or miss,...",pretty good their bread is kinda hit or miss i...
...,...,...,...,...,...,...
829,zQB2BH0NRx8r8kvDQXZ8mQ,Jimmy John's,40715 US Hwy 19 N,4.0,"My first time into a Jimmy Johns, and I'm glad...",my first time into a jimmy johns and i'm glad ...
830,zRjpagNQNZoovYfnmKxYmw,Subway,"1084 E Lancaster Ave, Ste 1",3.0,I've gotten a few sandwiches from here. It's ...,i've gotten a few sandwiches from here it's p...
831,zciO1XFzcKlX98Bi3XOGww,Subway,"3901 N Belt W, Ste B",1.5,HORRIBLE!! GO TO ANOTHER SUBWAY NOT THIS ONE!...,horrible go to another subway not this one i...
832,zqsmGvH1aPMDFJB-jFNPwQ,Jimmy John's,15069 Manchester Rd,1.5,After 30 minutes I had no food. Called up ther...,after 30 minutes i had no food called up there...


In [46]:
# remove stopwords
# DONT REMOVE STOPWORDS FOR BIGRAMS
def remove_mystopwords(sentence):
    tokens = sentence.split(" ")
    tokens_filtered= [word for word in tokens if not word in stopwords]
    return (" ").join(tokens_filtered)

def stopwords_list(df):
    res = []
    for sent in df['text_processed']:
        res.append(remove_mystopwords(sent))
    return res

In [50]:
# stop_removed = stopwords_list(df_merge)

In [51]:
# stop_removed

["repeated order three times loudly still got wrong that's fine sudden became rude aggressive pointed excuse besides pretty good job telling exactly opposite everyone else behind glass keeping rolling eyes me\n\nthis level customer service eye-opening",
 'typical subway one particularly close home convenient need quick dinner want skip greasy fast food alot options people watching weight also quite opposite ive never issues food customer service ill keep coming long stays way',
 'went tonight nice experience mark sandwich artist made us couple good subs efficient wanted tip learn owner decided allow hope paying reasonable rate mark deserves',
 "one bite i'm hooked friendly energetic staff made pleasure soon tasted sub (turkey provolone 'mike's way' hot peppers) knew found one favorite local places eat",
 'pretty good bread kinda hit miss ive gotten 5-6 sandwiches last months kinda stale bread :/ cheap though guess cant complain much\n\nplus bring us free party platers sandwiches someti

In [53]:
# tokenization and stemming/lemmatizing

from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 

stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()


def tokenization_and_lemmatizing(text):
    
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
            
    # stemming
    # stems = [stemmer.stem(t) for t in filtered_tokens]
    stems = [lemmatizer.lemmatize(t) for t in filtered_tokens]
    return stems

def list_tokenize_lemm(lst):
    res = []
    for sent in lst:
        res.append(tokenization_and_lemmatizing(sent))
    return res

In [54]:
processed_txt = list_tokenize_lemm(stop_removed)

In [55]:
processed_txt

[['repeated',
  'order',
  'three',
  'time',
  'loudly',
  'still',
  'got',
  'wrong',
  'fine',
  'sudden',
  'became',
  'rude',
  'aggressive',
  'pointed',
  'excuse',
  'besides',
  'pretty',
  'good',
  'job',
  'telling',
  'exactly',
  'opposite',
  'everyone',
  'else',
  'behind',
  'glass',
  'keeping',
  'rolling',
  'eye',
  'level',
  'customer',
  'service'],
 ['typical',
  'subway',
  'one',
  'particularly',
  'close',
  'home',
  'convenient',
  'need',
  'quick',
  'dinner',
  'want',
  'skip',
  'greasy',
  'fast',
  'food',
  'alot',
  'option',
  'people',
  'watching',
  'weight',
  'also',
  'quite',
  'opposite',
  'ive',
  'never',
  'issue',
  'food',
  'customer',
  'service',
  'ill',
  'keep',
  'coming',
  'long',
  'stay',
  'way'],
 ['went',
  'tonight',
  'nice',
  'experience',
  'mark',
  'sandwich',
  'artist',
  'made',
  'u',
  'couple',
  'good',
  'sub',
  'efficient',
  'wanted',
  'tip',
  'learn',
  'owner',
  'decided',
  'allow',
  'hope'

In [18]:
merge_review = []
for sent in df_review['text_processed']:
    merge_review.append(sent)

In [19]:
merge_review

["i repeated my order three times loudly and you still got it wrong that's fine but all of a sudden you became rude and aggressive after i pointed it out excuse me besides you did a pretty good job in telling exactly the opposite to everyone else behind the glass and keeping rolling your eyes on me\n\nthis level of customer service is eye-opening",
 'just your typical subway this one is particularly close to home and very convenient if i need a quick dinner but want to skip the greasy fast food they have alot of options for people watching their weight and also quite the opposite ive never had any issues with food or customer service here ill keep coming as long as it stays that way',
 'went here tonight and had a very nice experience mark the sandwich artist made us a couple of very good subs and was very efficient i wanted to tip him to learn that the owner decided not to allow this i hope they are paying a reasonable rate mark deserves it',
 "one bite and i'm hooked friendly energet

## TD-IDF Vectorizer

Example code for using the sklearn module:

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?'
]
# corpus = pd.DataFrame(corpus)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()
# array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
#        'this'], ...)
print(X.shape)

(4, 9)


(4, 9)

Our punctuation-removed corpus:

In [21]:
# USE THIS CODE FOR DATA FRAME
vect = TfidfVectorizer(ngram_range = (1, 2))
tfidf_matrix = vect.fit_transform(merge_review)
print(tfidf_matrix.shape)
df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names_out())

(834, 38109)


In [22]:
df

,00,00 bars,00 it,00 pm,00 when,000,000 sale,02,02 and,05,...,zero if,zero stars,zero tomatoes,zionsville,zionsville does,zip,zip around,zip right,zone,zone they
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
vect.get_feature_names_out()

array(['00', '00 bars', '00 it', ..., 'zip right', 'zone', 'zone they'],
      dtype=object)

## N-Grams

In [23]:
from nltk import ngrams

# sentence = input("Enter the sentence: ")
# n = int(input("Enter the value of n: "))
lst_grams = []
sentence = 'My husband and I ordered sandwiches for pickup today, which we do a couple times a month. His pastrami was well-made with a good portion of deli meat. My tuna... well at first I thought they entirely forgot the tuna! Pulled it apart to find a tiny spoonful on each half-- absolutely laughable amount of filling! I order this sandwich regularly and it\'s NEVER like that. He took it back and asked for a remake,l. The assistant managers said "it\'s our portion size", and never apologized or offered either a refund or a remake. So $7 for a teaspoon of tuna, and apathetic response from the employees... you ARE KIDDING ME!! Customer service standards and quality need an overhaul in this location.'
n_grams = ngrams(sentence.split(), 2)
for grams in n_grams:
    lst_bigrams.append(grams)

In [25]:
n_grams

<generator object ngrams at 0x000002905BB32548>

With review sentences:

In [24]:
corpus = ','.join(merge_review)

In [25]:
corpus

'i repeated my order three times loudly and you still got it wrong that\'s fine but all of a sudden you became rude and aggressive after i pointed it out excuse me besides you did a pretty good job in telling exactly the opposite to everyone else behind the glass and keeping rolling your eyes on me\n\nthis level of customer service is eye-opening,just your typical subway this one is particularly close to home and very convenient if i need a quick dinner but want to skip the greasy fast food they have alot of options for people watching their weight and also quite the opposite ive never had any issues with food or customer service here ill keep coming as long as it stays that way,went here tonight and had a very nice experience mark the sandwich artist made us a couple of very good subs and was very efficient i wanted to tip him to learn that the owner decided not to allow this i hope they are paying a reasonable rate mark deserves it,one bite and i\'m hooked friendly energetic staff th

In [26]:
from nltk import ngrams
lst_bigrams = []
n_grams = ngrams(corpus.split(), 2)
for grams in n_grams:
    lst_bigrams.append(grams)

In [27]:
lst_bigrams

[('i', 'repeated'),
 ('repeated', 'my'),
 ('my', 'order'),
 ('order', 'three'),
 ('three', 'times'),
 ('times', 'loudly'),
 ('loudly', 'and'),
 ('and', 'you'),
 ('you', 'still'),
 ('still', 'got'),
 ('got', 'it'),
 ('it', 'wrong'),
 ('wrong', "that's"),
 ("that's", 'fine'),
 ('fine', 'but'),
 ('but', 'all'),
 ('all', 'of'),
 ('of', 'a'),
 ('a', 'sudden'),
 ('sudden', 'you'),
 ('you', 'became'),
 ('became', 'rude'),
 ('rude', 'and'),
 ('and', 'aggressive'),
 ('aggressive', 'after'),
 ('after', 'i'),
 ('i', 'pointed'),
 ('pointed', 'it'),
 ('it', 'out'),
 ('out', 'excuse'),
 ('excuse', 'me'),
 ('me', 'besides'),
 ('besides', 'you'),
 ('you', 'did'),
 ('did', 'a'),
 ('a', 'pretty'),
 ('pretty', 'good'),
 ('good', 'job'),
 ('job', 'in'),
 ('in', 'telling'),
 ('telling', 'exactly'),
 ('exactly', 'the'),
 ('the', 'opposite'),
 ('opposite', 'to'),
 ('to', 'everyone'),
 ('everyone', 'else'),
 ('else', 'behind'),
 ('behind', 'the'),
 ('the', 'glass'),
 ('glass', 'and'),
 ('and', 'keeping'),
 ('

In [28]:
bigram_frequency = {}
for char in lst_bigrams:
    if char in bigram_frequency:
        bigram_frequency[char] += 1
    else:
        bigram_frequency[char] = 1
print(bigram_frequency)

{('i', 'repeated'): 1, ('repeated', 'my'): 1, ('my', 'order'): 65, ('order', 'three'): 1, ('three', 'times'): 2, ('times', 'loudly'): 1, ('loudly', 'and'): 2, ('and', 'you'): 18, ('you', 'still'): 3, ('still', 'got'): 1, ('got', 'it'): 4, ('it', 'wrong'): 1, ('wrong', "that's"): 1, ("that's", 'fine'): 2, ('fine', 'but'): 4, ('but', 'all'): 2, ('all', 'of'): 22, ('of', 'a'): 28, ('a', 'sudden'): 3, ('sudden', 'you'): 1, ('you', 'became'): 1, ('became', 'rude'): 1, ('rude', 'and'): 18, ('and', 'aggressive'): 1, ('aggressive', 'after'): 1, ('after', 'i'): 11, ('i', 'pointed'): 3, ('pointed', 'it'): 1, ('it', 'out'): 9, ('out', 'excuse'): 1, ('excuse', 'me'): 1, ('me', 'besides'): 1, ('besides', 'you'): 1, ('you', 'did'): 3, ('did', 'a'): 3, ('a', 'pretty'): 6, ('pretty', 'good'): 9, ('good', 'job'): 1, ('job', 'in'): 1, ('in', 'telling'): 1, ('telling', 'exactly'): 1, ('exactly', 'the'): 1, ('the', 'opposite'): 2, ('opposite', 'to'): 1, ('to', 'everyone'): 1, ('everyone', 'else'): 2, ('el

In [29]:
# sort by descending count
bigram_frequency_sorted = sorted(bigram_frequency.items(), 
                              key = lambda kv: kv[1], 
                              reverse = True)
type(bigram_frequency_sorted)

list

In [96]:
# save to csv upload to github
with open('bigram_counts.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    # for key, value in enumerate(bigram_frequency_sorted):
    writer.writerow(fields)

NameError: name 'fields' is not defined

In [30]:
bigram_frequency_sorted.to_csv('bigram_counts.csv') 

AttributeError: 'list' object has no attribute 'to_csv'

In [31]:
bigram_df = pd.DataFrame(bigram_frequency_sorted)
bigram_df.columns = ['bigrams', 'counts']

In [32]:
bigram_df.to_csv('bigram_counts.csv')

## Linear Regression

In [62]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import classification_report
import scipy.stats as stats

In [34]:
# merge tf-idf and business data
# df = tf-idf matrix
pd.concat([df_review, df], axis=1)

,business_id,name,address,stars_x,text,text_processed,00,00 bars,00 it,00 pm,...,zero if,zero stars,zero tomatoes,zionsville,zionsville does,zip,zip around,zip right,zone,zone they
0,#NAME?,Subway,1 Brookings Dr,2.0,I repeated my order three times loudly and you...,i repeated my order three times loudly and you...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-2CPhK6ik9ZBgFX_F-dkxQ,Subway,"14961 N Florida Ave, Ste 14961",2.0,Just your typical subway. This one is particul...,just your typical subway this one is particula...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-7UDKbg_8TL4LVuYR6Ooyw,Subway,"2451 Mcmullen Booth Rd, Ste 1",3.0,Went here tonight and had a very nice experien...,went here tonight and had a very nice experien...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-bcnY0PELKRomT2z7eGQ3w,Jersey Mike's Subs,575 Horsham Rd,3.5,One bite and I'm hooked! Friendly energetic st...,one bite and i'm hooked friendly energetic sta...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-dEuJqTHuE6rv8MrndqoXA,Jimmy John's,659 W Brandon Blvd,3.0,"Pretty good, their bread is kinda hit or miss,...",pretty good their bread is kinda hit or miss i...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,zQB2BH0NRx8r8kvDQXZ8mQ,Jimmy John's,40715 US Hwy 19 N,4.0,"My first time into a Jimmy Johns, and I'm glad...",my first time into a jimmy johns and i'm glad ...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
830,zRjpagNQNZoovYfnmKxYmw,Subway,"1084 E Lancaster Ave, Ste 1",3.0,I've gotten a few sandwiches from here. It's ...,i've gotten a few sandwiches from here it's p...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
831,zciO1XFzcKlX98Bi3XOGww,Subway,"3901 N Belt W, Ste B",1.5,HORRIBLE!! GO TO ANOTHER SUBWAY NOT THIS ONE!...,horrible go to another subway not this one i...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
832,zqsmGvH1aPMDFJB-jFNPwQ,Jimmy John's,15069 Manchester Rd,1.5,After 30 minutes I had no food. Called up ther...,after 30 minutes i had no food called up there...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df_no0 = df.drop(columns=df.columns[df.eq(0).mean()>0.9]) # dropping columns that are > 90% 0s

In [36]:
df_no0

,about,all,also,always,an,and,and the,and they,any,are,...,we,were,what,when,which,will,with,would,you,your
0,0.0,0.043036,0.000000,0.0,0.0,0.059113,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.10142,0.049705
1,0.0,0.000000,0.055092,0.0,0.0,0.040879,0.0,0.000000,0.055856,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.035126,0.000000,0.00000,0.051559
2,0.0,0.000000,0.000000,0.0,0.0,0.047150,0.0,0.000000,0.000000,0.042276,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000
3,0.0,0.000000,0.000000,0.0,0.0,0.077041,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.044133,0.000000,0.00000,0.000000
4,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.041456,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,0.0,0.000000,0.000000,0.0,0.0,0.073712,0.0,0.080296,0.040288,0.079311,...,0.0,0.0,0.069967,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000
830,0.0,0.051883,0.000000,0.0,0.0,0.023755,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000
831,0.0,0.000000,0.000000,0.0,0.0,0.058325,0.0,0.000000,0.053130,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000
832,0.0,0.000000,0.000000,0.0,0.0,0.023614,0.0,0.064307,0.000000,0.000000,...,0.0,0.0,0.056035,0.0,0.0,0.0,0.000000,0.057248,0.00000,0.000000


In [38]:
print(df_no0.columns)

Index(['about', 'all', 'also', 'always', 'an', 'and', 'and the', 'and they',
       'any', 'are',
       ...
       'we', 'were', 'what', 'when', 'which', 'will', 'with', 'would', 'you',
       'your'],
      dtype='object', length=112)


MLR with df_no0 data as x features:

In [125]:
x = df_no0
y = df_review['stars_x']

In [126]:
x, y = np.array(x), np.array(y)

In [127]:
model = LinearRegression().fit(x, y)

In [128]:
r_sq = model.score(x, y)

In [129]:
print(f"intercept: {model.intercept_}")

intercept: 2.738949396383648


In [131]:
print(f"coefficients: {model.coef_}")
df_no0.columns

coefficients: [ 2.54399834  0.10055339 -0.91781998  3.28821594 -1.47221783  1.07264982
  0.12663893  1.28546935  0.44426507 -0.36295139  1.10810419  1.99867756
  0.06306843  0.84905136  1.77691658 -1.03166284  1.72480452 -0.25791568
 -1.04795867  0.99831812  0.62785901 -0.86776899 -2.20898492 -0.25874318
 -1.62865826 -0.38985839  0.08884171  1.05138595 -1.86955455  0.59491393
  1.54527822 -0.44409716  0.63158148  2.12927961  1.99192723 -2.25885983
 -0.06306885  0.33204643  0.81002823 -0.90876077]


Index(['also', 'always', 'back', 'better', 'bread', 'clean', 'employees',
       'even', 'fast', 'food', 'fresh', 'friendly', 'get', 'go', 'good', 'got',
       'great', 'like', 'location', 'love', 'lunch', 'make', 'never', 'one',
       'order', 'ordered', 'people', 'place', 'really', 'sandwich',
       'sandwiches', 'service', 'staff', 'sub', 'subs', 'subway', 'time', 've',
       'way', 'would'],
      dtype='object')

MLR with all 30,000+ unigrams & bigrams:

In [64]:
x = df
y = df_review['stars_x']

In [65]:
x, y = np.array(x), np.array(y)

In [109]:
model = LinearRegression().fit(x, y)

In [110]:
r_sq = model.score(x, y)
r_sq

1.0

In [111]:
print(f"intercept: {model.intercept_}")

intercept: 2.824132603377369


In [112]:
print(f"coefficients: {model.coef_}")

coefficients: [-0.13376532 -0.09220681 -0.15799626 ...  0.12778548  0.1016731
  0.10174769]


In [118]:
max(model.coef_)

0.991794369709788

MLR choosing single words and n-grams:

In [181]:
df_select = df[["bread", "staff", "beef", "lunch", "cheese", "fresh", "fast", 
                "flatbread", "teriyaki", 'meatball', 'italian', 'tomato', 'steak']]

In [182]:
df_select

,bread,staff,beef,lunch,cheese,fresh,fast,flatbread,teriyaki,meatball,italian,tomato,steak
0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.048758,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.000000,0.057542,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.118031,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.035168,0.0,0.0,0.000000,0.0,0.0,0.0
830,0.060642,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
831,0.000000,0.000000,0.0,0.0,0.058083,0.0,0.000000,0.0,0.0,0.075066,0.0,0.0,0.0
832,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


In [183]:
x = df_select
y = df_review['stars_x']

In [212]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size= 834 - train_samples, random_state = 11) # 80/20 split

In [213]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [214]:
model = LinearRegression().fit(X_train, y_train)

In [215]:
r_sq = model.score(x, y)
r_sq

0.0254196642406872

In [187]:
print(f"intercept: {model.intercept_}")

intercept: 2.7744311071914116


In [188]:
print(f"coefficients: {model.coef_}")

coefficients: [-1.54970724  2.14517492  2.87647606  0.44702267  1.07377112  1.30268824
  1.6430803  -1.64876142 -2.42673023 -0.29871609  2.15272223  2.69492333
  2.21383941]


2.876476063962305

In [216]:
yhat = model.predict(X_test)

C:\Users\sebre\Anaconda3\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [221]:
yhat = np.around(yhat, decimals = 1)

In [224]:
yhat = yhat.astype(int)

In [226]:
score = accuracy_score(yhat, y_test)
score

0.41317365269461076

Using statsmodels:

In [57]:
from numpy import ravel

In [190]:
# X = sm.add_constant(X.ravel())
results = sm.OLS(y,x).fit()
results.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.414
Model:                            OLS   Adj. R-squared (uncentered):              0.404
Method:                 Least Squares   F-statistic:                              44.56
Date:                Wed, 07 Dec 2022   Prob (F-statistic):                    4.50e-86
Time:                        15:32:09   Log-Likelihood:                         -1859.3
No. Observations:                 834   AIC:                                      3745.
Df Residuals:                     821   BIC:                                      3806.
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            14.7634      3.115      4.739      0.000       8.648      20.879
x2            29.1161      3.029      9.612      0.000      23.171      35.062
x3            19.0548      5.297      3.597      0.000       8.657      29.453
x4            29.5540      4.213      7.014      0.000      21.284      37.824
x5            17.1216      4.588      3.731      0.000       8.115      26.128
x6            16.3479      3.667      4.458      0.000       9.151      23.545
x7            23.0978      3.231      7.148      0.000      16.755      29.440
x8            14.9692      7.023      2.132      0.033       1.184      28.754
x9            16.2827      8.189      1.988      0.047       0.209      32.357
x10            8.6391      5.228      1.653      0.099      -1.622      18.901
x11           19.5901      4.897      4.000      0.000       9.978      29.202
x12           21.5525      7.502      2.873      0.004       6.827      36.278
x13            3.1466      4.731      0.665      0.506      -6.139      12.432
==============================================================================
Omnibus:                      216.453   Durbin-Watson:                   0.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              824.668
Skew:                          -1.182   Prob(JB):                    8.43e-180
Kurtosis:                       7.259   Cond. No.                         3.53
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
y_preds = reg.predict(X_test)

print(classification_report(y_test,y_preds)

In [82]:
len(x)

834

## Multinomial Logistic Regression

In [208]:
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state
from sklearn.utils.multiclass import type_of_target
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

prep data train & test sets

In [191]:
train_samples = 667
X = df_select
y = df_review['stars_x']

# type_of_target(y)
# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(y)
y = y.astype('int')
type_of_target(y)

'multiclass'

In [202]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size= 834 - train_samples, random_state = 11) # 80/20 split

In [203]:
y_test

311    2
307    2
73     3
662    2
663    3
      ..
158    3
424    3
177    1
588    3
803    3
Name: stars_x, Length: 167, dtype: int32

model accuracy with CV:

In [211]:
# define the multinomial logistic regression model with a default penalty
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                           C=1.0, max_iter = 1000000)
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

C:\Users\sebre\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,
C:\Users\sebre\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,
C:\Users\sebre\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,


Mean Accuracy: 0.424 (0.006)


In [205]:
# predictions
result = model.fit(X, y)

In [206]:
y_test

311    2
307    2
73     3
662    2
663    3
      ..
158    3
424    3
177    1
588    3
803    3
Name: stars_x, Length: 167, dtype: int32

In [196]:
row = X_test# .iloc[0:1, :] # first row of test data
# predict a multinomial probability distribution
p = model.predict_proba(row)
# summarize the predicted probabilities
print(p) # 'Predicted Probabilities: %s' %

[[0.08339933 0.37371348 0.42607607 0.11561287 0.00119825]
 [0.0832175  0.37102581 0.41567426 0.12886349 0.00121894]
 [0.08129487 0.40178324 0.40400011 0.11173851 0.00118327]
 [0.08339933 0.37371348 0.42607607 0.11561287 0.00119825]
 [0.08267225 0.37082706 0.4255227  0.11977567 0.00120232]
 [0.08201265 0.37928897 0.41302325 0.12446615 0.00120899]
 [0.08276312 0.37635236 0.42001212 0.11966899 0.00120341]
 [0.08339933 0.37371348 0.42607607 0.11561287 0.00119825]
 [0.08339933 0.37371348 0.42607607 0.11561287 0.00119825]
 [0.08339933 0.37371348 0.42607607 0.11561287 0.00119825]
 [0.08122548 0.37852259 0.42505207 0.11401209 0.00118776]
 [0.08339933 0.37371348 0.42607607 0.11561287 0.00119825]
 [0.08211716 0.37719801 0.42149711 0.11799016 0.00119755]
 [0.08339933 0.37371348 0.42607607 0.11561287 0.00119825]
 [0.08339933 0.37371348 0.42607607 0.11561287 0.00119825]
 [0.08339933 0.37371348 0.42607607 0.11561287 0.00119825]
 [0.08339933 0.37371348 0.42607607 0.11561287 0.00119825]
 [0.08208984 0

In [197]:
# predict the class label
yhat = model.predict(row)
print(yhat) # 'Predicted Class: %d' %

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [199]:
print(result.intercept_)
print(result.coef_)

[ 0.15706831  1.65691748  1.78804599  0.48367537 -4.08570715]
[[-0.04279623 -0.28340552 -0.12869723 -0.11967239 -0.02052533 -0.25027813
  -0.14276852 -0.06320074  0.00274854 -0.03006891 -0.20130633 -0.00241783
  -0.22992131]
 [ 0.39933147  0.05183566 -0.15087477  0.09389111 -0.329255   -0.07988118
  -0.13171245  0.27146058  0.13797558  0.03832293 -0.14784041 -0.13312741
  -0.16101296]
 [-0.23586554 -0.43573265  0.03150815  0.04025376 -0.16267839  0.32209256
  -0.06054342 -0.11885882 -0.06870347 -0.25387374  0.28288756  0.04864036
   0.03900715]
 [-0.11222276  0.67800479  0.24994987 -0.00879915  0.51789987  0.01588277
   0.34385478 -0.08848282 -0.07127564  0.24800934  0.06922809  0.08850306
   0.35535794]
 [-0.00844693 -0.01070228 -0.00188602 -0.00567333 -0.00544115 -0.00781602
  -0.00883039 -0.00091819 -0.00074501 -0.00238961 -0.0029689  -0.00159818
  -0.00343083]]


In [200]:
summary = pd.DataFrame(zip(X.columns, np.transpose(result.coef_.tolist()[0])), 
                       columns=['features', 'coef'])

In [201]:
print(summary)

     features      coef
0       bread -0.042796
1       staff -0.283406
2        beef -0.128697
3       lunch -0.119672
4      cheese -0.020525
5       fresh -0.250278
6        fast -0.142769
7   flatbread -0.063201
8    teriyaki  0.002749
9    meatball -0.030069
10    italian -0.201306
11     tomato -0.002418
12      steak -0.229921


In [210]:
# test set accuracy 
score = accuracy_score(yhat, y_test)
score

0.4251497005988024